In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from typing import Tuple
from sklearn import preprocessing

In [43]:
def load_data() -> Tuple[np.ndarray, np.ndarray]:
    data = pd.read_csv("./data/data.csv").drop(["Unnamed: 0"], axis=1)
    labels = pd.read_csv("./data/labels.csv")["Class"]
        
    # Encodes string labels as ints
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(labels)
    
    return data.to_numpy(), y
X, y = load_data()

# First feature reduction to reduce run time

In [81]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif

# Dropping the constant features
X_nonconst = VarianceThreshold().fit_transform(X)

# Doing feature selection by choosing the k features with the largest F-statistic
k = 200
X_fstat = SelectKBest(score_func=f_classif, k=k).fit_transform(X_nonconst, y)

# Feature selection using multi-class logistic regression

In [169]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import balanced_accuracy_score as bacc

def cross_validate(n_folds, C_list, X, y):
    """
    Cross-validates the parameter values in C_list and
    returns a numpy.ndarray where the first index corresponds to the
    different values in C_list and the columns correspond
    to the different folds. Each of the elements has the balanced
    accuracy score for a given parameter value adn fold.
    """
    score_matrix = np.zeros((len(C_list), n_folds))
    skf = StratifiedKFold(n_splits=n_folds)
    ix = 0
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        jx = 0
        for val in C_list:
            mdl = LogisticRegression(C=val,
                                    multi_class='ovr', 
                                    solver='liblinear',
                                    intercept_scaling=10000).fit(X_train, y_train)
            y_pred = mdl.predict(X_test)
            score_matrix[jx, ix] = bacc(y_test, y_pred)
            
            jx += 1
        ix += 1
        
    return score_matrix

In [171]:
n_folds = 10
c_list = np.logspace(-4, 4, 30)

score_matrix = cross_validate(10, c_list, X_fstat, y)
cv_mean = np.mean(score_matrix, axis=1)
cv_std = np.std(score_matrix, axis=1)
idx_max_mean = np.argmax( cv_mean )
c_max = c_list[idx_max_mean]
idx_c_1se = np.where(
    (cv_mean <= cv_mean[idx_max_mean] + cv_std[idx_max_mean] / np.sqrt(n_folds)) &
    (cv_mean >= cv_mean[idx_max_mean])
)[0][0]
c_1se = c_list[idx_c_1se]


# TODO: we get score for every different class. How do we handle this to 
# find c_1se?
# Finds the C_1se for each label, i.e. the smalles C that produces
# score within one std of the mean score for C_min for a given label
#c_1se = {}
#for key, val in log_reg.scores_.items():
#    cv_mean = np.mean(val, axis=1)
#    cv_std = np.std(val, axis=1)
#    idx_max_mean = np.argmax(cv_mean)
#    idx_c = np.where(
#        (cv_mean >= cv_mean[idx_max_mean] - cv_std[idx_max_mean] / np.sqrt(n_folds)) &
#        (cv_mean <= cv_mean[idx_max_mean])
#    )[0][0]    
#    c_1se[key] = log_reg.C_[idx_c]

NameError: name 'score_matrix_' is not defined

# Gaining confidence in feature selection thorugh bootstrap

In [157]:
from sklearn.linear_model import LogisticRegression


n_bootstrap_samples = 100
sample_size = 150

# Creating bootstrap sample and records how often a given feature is 
# selected in the LogisticRegression for c_max.
selected_feature_counter = np.zeros(k)
for ix in range(n_bootstrap_samples):
    idx_list = np.random.choice(X_fstat.shape[1], sample_size, replace=False)
    X_bootstrap = X_fstat[:,idx_list]
    beta = LogisticRegressionCV(Cs=c_list,
                              cv=n_folds,
                             multi_class='ovr', 
                             solver='liblinear',
                             intercept_scaling=10000).fit(X_bootstrap, y).coef_
    _, non_zero_idx = np.where( abs(beta) == 0 )
    non_zero_orig_idx = idx_list[non_zero_idx]
    
    for jx in non_zero_orig_idx:
        selected_feature_counter[jx] += 1
plt.bar(range(200), selected_feature_counter)


KeyboardInterrupt: 